In [ ]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import math
import math
from scipy import interpolate
from cellpose import plot, utils
import os
from library import plots

## Read in Data

In [ ]:
dic = []
for file in os.listdir("brightfield/"):
    if not file.startswith('.'):
        first = cv2.imread(os.path.join("brightfield/", file), cv2.IMREAD_GRAYSCALE)
        dic.append(first)

red = []
for file in os.listdir("red/"):
    if not file.startswith('.'):
        first = cv2.imread(os.path.join("red/", file), cv2.IMREAD_GRAYSCALE)
        red.append(first)

blue = []
for file in os.listdir("blue/"):
    if not file.startswith('.'):
        first = cv2.imread(os.path.join("blue/", file), cv2.IMREAD_GRAYSCALE)
        blue.append(first)

## Create segmentations of blue and red

In [ ]:
# RUN CELLPOSE
import os
from cellpose import models, io
from cellpose import plot

# DEFINE CELLPOSE MODEL
model = models.CellposeModel(gpu=False, model_type='LC4')

chan = [0,0]
for idx,img in enumerate(dic):
    masks, flows, diams = model.eval(img, diameter=34.9,flow_threshold=0.4, channels=chan)
    io.masks_flows_to_seg(img, masks, flows, 34.9, f'segmentations/{idx}', chan)

## Example segmentation images

In [ ]:
img = dic[0]
masks, flows, styles = model.eval(img, diameter=45.6,flow_threshold=0.4, channels=chan)
fig = plt.figure(figsize=(12,5))
plot.show_segmentation(fig, img, masks, flows[0], channels=chan)
plt.tight_layout()
plt.show()

## Use segmentations to create plots

#### Find limits of data


In [ ]:
def get_cell_masks(img, outlines):
    masked_imgs= []
    for o in outlines:
        x = o[:,0]
        y = o[:,1]
        x = np.r_[x, x[0]]
        y = np.r_[y, y[0]]
        tck, u = interpolate.splprep([x, y], s=0, per=True)
        xi, yi = interpolate.splev(np.linspace(0, 1, 1000), tck)
        contour = np.array([[xii, yii] for xii, yii in zip(xi.astype(int), yi.astype(int))])
        mask    = np.zeros_like(img)
        cv2.fillPoly(mask, pts=[contour], color=(255, 255, 255))
        masked_img = cv2.bitwise_and(img, mask)
        masked_imgs.append(masked_img)
    return masked_imgs

def get_pixel_intensities(img, pts):
    pixel_intensities = img[pts[0], pts[1]] 
    return pixel_intensities

def xy_lim(file):
    # red limit
    dat = np.load(f'red/{file}_seg.npy', allow_pickle=True).item()
    img = dat['img']
    outlines = utils.outlines_list(dat['masks'])
    masked_imgs_red = get_cell_masks(img,outlines)
    means_red = np.array([np.mean(get_pixel_intensities(img, np.where(masked_img > 0))) for masked_img in masked_imgs_red])

    # blue limit
    dat = np.load(f'blue/{file}_seg.npy', allow_pickle=True).item()
    img = dat['img']
    masked_imgs_blue = get_cell_masks(img,outlines)
    means_blue = np.array([np.mean(get_pixel_intensities(img, np.where(masked_img > 0))) for masked_img in masked_imgs_blue])

    return max((np.max(means_red),np.max(means_blue)))

max_limit = 0
for idx in range(len(blue)):
    max_limit = max(max_limit, xy_lim(f'{idx}'))
print(max_limit)

#### Scatter Plot

In [ ]:
def blue_red_cell_scatter(file,data):
    font = {
        'weight' : 'bold',
        'size'   : 22
        }
    plt.rc('font', **font)
    dat = np.load(f'dic/{file}_seg.npy', allow_pickle=True).item()
    # red limit
    img = data['red'][file]
    outlines = utils.outlines_list(dat['masks'])
    masked_imgs_red = get_cell_masks(img,outlines)
    means_red = np.array([np.mean(get_pixel_intensities(img, np.where(masked_img > 0))) for masked_img in masked_imgs_red])

    # blue limit
    img = data['red'][file]
    masked_imgs_blue = get_cell_masks(img,outlines)
    means_blue = np.array([np.mean(get_pixel_intensities(img, np.where(masked_img > 0))) for masked_img in masked_imgs_blue])

    # plot image with masks overlaid
    mask_RGB = plot.mask_overlay(dat['img'], dat['masks'])

    # plot image with outlines overlaid in red (this is for blue segmentation)
    outlines = utils.outlines_list(dat['masks'])
    plt.figure(figsize=(12,10))
    plt.imshow(dat['img'])
    for o in outlines:
        plt.plot(o[:,0], o[:,1], color='r')
    
    # scatter plot
    plt.plot(means_red, means_blue, 'o')
    plt.xlabel('Red')
    plt.ylabel('Blue')
    plt.gca().set_ylim(-0.5, data['lim'])
    plt.gca().set_xlim(-0.5, data['lim'])
    plt.title(f'Blue vs Red Intensity at frame {file}')

    #  intrinsic and extrinsic noise
    points_blue = means_blue
    points_red = means_red
    points_blue_mean = np.mean(points_blue)
    points_red_mean = np.mean(points_red)
    intrinsic = (1/len(points_blue))*sum([0.5*(blue - red)**2 for blue, red in zip(points_blue, points_red)])/(points_blue_mean * points_red_mean)
    extrinsic = (1/len(points_blue))*sum([blue*red - points_red_mean*points_blue_mean for blue, red in zip(points_blue, points_red)])/(points_blue_mean * points_red_mean)
    data["intrinsics"].append(intrinsic)
    data["extrinsics"].append(extrinsic)
    data["cv_red"].append(np.std(means_red)/np.mean(means_red))
    data["cv_blue"].append(np.std(means_blue)/np.mean(means_blue))

In [ ]:
name_of_file = 'plots/cell_scatter_wimg.gif'
plotting_function = blue_red_cell_scatter
intrinsics = []
extrinsics = []
cv_red = []
cv_blue = []
data = {}
data['intrinsics'] = intrinsics
data['extrinsics'] = extrinsics
data["cv_red"] = cv_red
data["cv_blue"] = cv_blue
data["lim"] = max_limit
plots.create_gif(name_of_file, plotting_function, 0.0, red, data, no_images=True)

#### Example bar plot

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
from cellpose import plot, utils
list_of_cv = []
def plot_hist(file,color_img):
    dat= np.load(f'{file}_seg.npy', allow_pickle=True).item()
    # plot image with outlines overlaid in red
    img = dat['img']
    outlines = utils.outlines_list(dat['masks'])
    masked_imgs = get_cell_masks(img,outlines)
    means = np.array([np.mean(get_pixel_intensities(color_img, np.where(masked_img > 0))) for masked_img in masked_imgs])
    # Exclude outliers in the data
    def reject_outliers(data, m=1):
        return data[abs(data - np.mean(data)) < m * np.std(data)]
    means = reject_outliers(means)

    # plot image with masks overlaid
    mask_RGB = plot.mask_overlay(img, dat['masks'])

    # plot image with outlines overlaid in red
    plt.figure(figsize=(12,10))
    plt.imshow(img)
    for o in outlines:
        plt.plot(o[:,0], o[:,1], color='r')

    plt.figure()
    # Plot the mean intensity of each cell
    plt.hist(means)
    plt.xlabel('Mean intensity')
    plt.ylabel('Cell number')
    plt.show()

file = 12
plot_hist(f'dic/{file}',red[file])